# Chapter 10: Sequence Modelling

ConvNets based on ideas of parameter sharing, and sparse connections succeeded in various tasks specific to images. It makes intuitive sense to extend the core idea of parameter sharing to represent sequential data. Consider two specific sequences that frequently occur in natural language processing.

1.   I went to Nepal in 2009.
2.   In 2009, I went to Nepal.

We would like capture the sequential information to get a vector representation of such sequences. Ordinary bag-of-words models can not properly capture the positional information in a sentence. Recurrent Neural networks (RNN) extend this idea of learning dependencies between words efficiently using backpropagation, parameter sharing and using various hacks around it.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import keras

Using TensorFlow backend.


## Recurrent Neural Networks

* The ideal way to think about RNN's is a computational graph. The state of the network at a given postion is a function of the input at that position and feedback from the previous position. 
* The weights are shared among the entire length of sequence. An  RNN will take a fixed length sequence and learns to represent it in a hidden state (h). 
* The hidden state contains a noisy lookup table of all the previous states, so the network has access to information about long range dependencies. 
* One can try to use something like a logistic regression at each position feeding it the output from previous sequences. But, it can not efficiently capture all the long range dependencies. On the other hand, shared parameter weights allow the RNN to generalize to unseen data.

### Forward Pass: Elman RNN

Here is the forward pass of a vanilla RNN. The input is mapped into a hidden state by using a function of previous state and current input. The mapped hidden state is then used to predict a probability over classes.

$$ Input\ to\ hidden: a^{(t)} = b + W h^{(t-1)} + U x^{(t)} $$

$$ Hidden\ activation: h^{(t)} = Tanh(a^{(t)}) $$

$$ Output\ state: o^{(t)} = c + V h^{(t)} $$

$$ Class\ probabilities: y^{(t)} = softmax(o^{(t)}) $$


### Backward pass:

During backward pass, one needs to compute the gradient of the loss with respect to the inputs. But in addition to that, we have a sequence that is coming in, each step contributing a loss. In the end, you have to backpropagate through the entire loss. This is called backpropagation through time (BPTT).

$$ Total\ loss\ for\ sequence\ length\ 1\ to\ \tau  = \sum L{(t)}$$

## Teacher forcing and RNN's with output recurrence:

* You can wire these RNN's in various ways. One other possible variant of the RNN does not have any recurrence at the hidden state, but at the output.
* At time t+1, the model has access to the input at time t+1 and previous output at t.
* This can be parellelized very efficiently, and can be trained using 'teacher forcing'.

* You can estimate the total probablity $ log P(y^{(1)}, y^{(2)} | x^{(1)}, x^{(2)}) $ as sum of two individual terms.
* At test time, we don't have outputs. So noisy estimate of current output is used to estimate the next output.

## Recurrent Networks as Directed Graphical Models:

* RNN's can be intrepreted as fully connected graphical models over the sequence of outputs. To put it more bluntly, every future state can be influenced by it's every past state.
* If we were to start learning this fully connected graphical model without RNN's, there would be too many connections and parameters. This is very inefficient.
* Another nice way to intrepret RNN's is to think of hidden units as random variables capturing distribution between outputs efficiently.
* Traditionally it is common to take something like a Markov asssumption, meaning that one output can only be influenced by using the previous input. This assumption is restrictive, although it works in things like speech recognition. RNN's can capture long range dependencies very efficiently if trained with enough capacity and carefully choosen parameters.
* This makes the optimization very tough though. But the amount of efficiency we get in the prediction is insanely high. Let's say we have to predict k classes on a sequence of length $ \tau $. By using a graphical model, the parameters grow with $ O(k^{(\tau)}) $. If we use an RNN, the output order becomes $ O(1) $ because we are sharing the same parameters at each time step, there by learning a good generalized function.

## Bidirectional RNN's:

* The caveat with RNN's is that they only capture the information along one direction.
* Sometimes, you might have access to the whole sequence (let's say in a text classification). It then makes sense to have an RNN read the sequence backwards as well to capture any context that is coming from the other direction.
* Practioners in Speech recognition, handwriting recognition and bioinformatics fields are using Bidirectional RNN's to improve their performance.
* The output is concatenated from the forward and the backward RNN.

#### Bidirectional RNN's on Images:

* This idea can be extended to two dimensional input as well.
* On every point in an image, you can use four RNN's to capture the contextual information coming from four direcitons. This helps the network to capture very long term dependencies on images as well.

## Encoder-Decoder Sequence to Sequence RNN's:


* In machine translation, and speech recognition, both input and output are sequences of variable length.
* In this case, an RNN could be used to encode the input sequence into a vector of fixed length and a decoder uses this vector to output the sequence.
* These networks are also trained to recognize the end of sequence and beginning of sequence as well. So, this process is completly end-to-end requiring no manual feature engineering.
* Machine translation models have been epitome of success of deep neural networks providing accurate performance across various languages.

## Deep Recurrent Networks:

* Is depth advantageous in recurrent networks? Experiments strongly suggest 'yes'.
* There are various ways of introducing depth in RNN's.
    1. Stack layers hierarchically
    2. A multi-layer perceptron between hidden to hidden connections instead of a direct single layer mapping.
    3. Skip connections between hidden layers leading to extra parameters

## Recursive Neural Networks:

* These networks are structured as a deep tree rather than a sequential chain like RNN.
* In addition to tree like structure, one can introduce a certain level of parameter sharing among layers. This allows the network to learn the structure.
* However, trying to learn the structure automatically is not very easy. Questions like what kind of tree is required needs to be answered.
* These networks can be useful in NLP, they can be forced to learn the structure that mimics the parse tree of a sentence. Doing this dynamically is a challenge.

## The challenge of capturing long-term dependencies: